In [108]:
import pandas as pd
import numpy as np
import json
import pprint
import pymongo
from pymongo import MongoClient

In [109]:
co_df = pd.read_csv('data/CO2_flat.csv')
pc_df = pd.read_csv('data/percapita_flat.csv')


In [110]:
# Adding identifier columns to the dataframes so that we can merge them later
# Assuming df_totals and df_percapita are your two data frames
co_df['Type'] = 'Total CO2'
pc_df['Type'] = 'PerCapita CO2'

In [111]:
# Drop the per capita column from the co_df as it will be maintained in the pc_df
co_df = co_df.drop(columns = ['Per Capita'])

In [112]:
# Look at nulls
co_df.isnull().sum()

# Where is the iso code missing?
co_df[co_df['ISO 3166-1 alpha-3'].isnull()]

,Country,ISO 3166-1 alpha-3,Year,Total,Coal,Oil,Gas,Cement,Flaring,Other,Type
19312,French Equatorial Africa,NaN,1750,0.0,NaN,NaN,NaN,NaN,NaN,NaN,Total CO2
19313,French Equatorial Africa,NaN,1751,0.0,NaN,NaN,NaN,NaN,NaN,NaN,Total CO2
19314,French Equatorial Africa,NaN,1752,0.0,NaN,NaN,NaN,NaN,NaN,NaN,Total CO2
19315,French Equatorial Africa,NaN,1753,0.0,NaN,NaN,NaN,NaN,NaN,NaN,Total CO2
19316,French Equatorial Africa,NaN,1754,0.0,NaN,NaN,NaN,NaN,NaN,NaN,Total CO2
...,...,...,...,...,...,...,...,...,...,...,...
47867,Ryukyu Islands,NaN,2017,0.0,NaN,NaN,NaN,NaN,NaN,NaN,Total CO2
47868,Ryukyu Islands,NaN,2018,0.0,NaN,NaN,NaN,NaN,NaN,NaN,Total CO2
47869,Ryukyu Islands,NaN,2019,0.0,NaN,NaN,NaN,NaN,NaN,NaN,Total CO2
47870,Ryukyu Islands,NaN,2020,0.0,NaN,NaN,NaN,NaN,NaN,NaN,Total CO2


In [113]:
# Look at nulls in the other dataframe - both CO2 dataframes have the same country codes and data missing
pc_df.isnull().sum()

Country                   0
ISO 3166-1 alpha-3     1632
Year                      0
Total                 44132
Coal                  45966
Oil                   46065
Gas                   46092
Cement                47656
Flaring               46160
Other                 61484
Type                      0
dtype: int64

In [114]:
# Drop the rows with null country codes
co_df = co_df.dropna(subset=['ISO 3166-1 alpha-3'])
pc_df = pc_df.dropna(subset=['ISO 3166-1 alpha-3'])

In [115]:
# Concatenate the data frames
all_co_df = pd.concat([co_df, pc_df], ignore_index=True)

# Melt the data frame, id_vars is what columns to keep, value_vars is what columns to melt
all_co_melted = pd.melt(all_co_df, id_vars=['Country', 'ISO 3166-1 alpha-3', 'Year', 'Type'],
                    value_vars=['Total', 'Coal', 'Oil', 'Gas', 'Cement', 'Flaring', 'Other'],
                    var_name='Category', value_name='Emission')

In [116]:
# Renaming columns for clarity
all_co_melted = all_co_melted.rename(columns = {'ISO 3166-1 alpha-3':'Country Code', 'Emission':'Value'})

all_co_melted.head()

,Country,Country Code,Year,Type,Category,Value
0,Afghanistan,AFG,1750,Total CO2,Total,0.0
1,Afghanistan,AFG,1751,Total CO2,Total,0.0
2,Afghanistan,AFG,1752,Total CO2,Total,0.0
3,Afghanistan,AFG,1753,Total CO2,Total,0.0
4,Afghanistan,AFG,1754,Total CO2,Total,0.0


In [117]:
# Shortening the time frame from 1999-2019
all_co_melted_short = all_co_melted[all_co_melted['Year'] >= 1999]

In [118]:
all_co_melted_short = all_co_melted_short.copy()

# Converting the data types to float
all_co_melted_short[['Value']] = all_co_melted_short[['Value']].astype(float)


In [119]:
# Find where there are World or International Transport values
# all_co_melted[all_co_melted['Country'] == 'World']
# all_co_melted[all_co_melted['Country'] == 'International transport']

# Drop the rows with World and International Transport values
all_co_ready = all_co_melted_short[all_co_melted['Country'] != 'World']
all_co_ready = all_co_melted_short[all_co_melted['Country'] != 'International transport']


/opt/anaconda3/envs/pythondata/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/opt/anaconda3/envs/pythondata/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys


In [120]:
all_co_ready.head()


,Country,Country Code,Year,Type,Category,Value
249,Afghanistan,AFG,1999,Total CO2,Total,1.091640
250,Afghanistan,AFG,2000,Total CO2,Total,1.047128
251,Afghanistan,AFG,2001,Total CO2,Total,1.069098
252,Afghanistan,AFG,2002,Total CO2,Total,1.340995
253,Afghanistan,AFG,2003,Total CO2,Total,1.559602


### Bringing in the GDP and Population data
These are from the same source, in the same format just pulled as different times

In [121]:
# Read in the GDP data
gdp_df= pd.read_csv('data/gdp_data.csv')
pop_df = pd.read_csv('data/population.csv')

# Concatenate the data frames
all_gdp_df = pd.concat([gdp_df, pop_df], ignore_index=True)

In [122]:
# Cleaning up the column names
new_columns = []
for column in all_gdp_df.columns:
    new_columns.append(column.split('[')[0].strip())

all_gdp_df.columns = new_columns

# Renaming the series to match Type from the CO2 data
all_gdp_df = all_gdp_df.rename(columns = {'Series Name':'Category', 'Country Name':'Country'})

# Adding the Type column for merging later with the CO2 data, add it in column 2
all_gdp_df.insert(2, 'Type', 'GDP')

# Dropping the series code column
all_gdp_df_formatted = all_gdp_df.drop(columns = ['Series Code'])

all_gdp_df_formatted.head()

,Country,Country Code,Type,Category,1960,1961,1962,1963,1964,1965,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Afghanistan,AFG,GDP,GDP (current US$),537777811.111111,548888895.555556,546666677.777778,751111191.111111,800000044.444444,1006666637.77778,...,15633856787.0425,18190410820.6235,20203572959.5023,20564485419.1684,20550582746.8448,19998156214.3988,18019558181.9602,18896352021.94,18418848299.5825,18904490262.913
1,Afghanistan,AFG,GDP,GDP per capita (current US$),62.3693745050559,62.4437034626929,60.9503638210144,82.0217375781519,85.5110734102311,105.243195716467,...,554.595200222354,621.912310861592,663.141052810937,651.987861948108,628.146803888496,592.476537451681,520.252064031151,530.149830802984,502.056770622973,500.522664145294
2,Africa Eastern and Southern,AFE,GDP,GDP (current US$),21291524631.3606,21809435284.9773,23708060554.431,28211280561.2905,26119938954.6996,29683481336.0513,...,860361207235.72,964213016259.692,972002199045.518,982677082467.187,1003403000510.52,923143900034.922,889859250365.411,1030482257898.25,1016696893390.38,1009051505055.59
3,Africa Eastern and Southern,AFE,GDP,GDP per capita (current US$),162.913034498773,162.551683028333,172.002459991871,199.189238183763,179.387798934375,198.230367947484,...,1643.60556870139,1792.90750512756,1759.18239469392,1730.39468555029,1719.18372097247,1538.55226805656,1443.69237090812,1628.58678812124,1564.73433998697,1512.2705529912
4,Africa Western and Central,AFW,GDP,GDP (current US$),10404135069.15,11127894641.0191,11943187848.3043,12676330764.6917,13838369295.2313,14862225759.9135,...,597129288864.775,680455985950.414,736039861278.13,832216894624.603,892497905712.366,766957955078.105,690545418736.157,683748014299.694,766359667820.703,794719102944.191


In [123]:
all_gdp_df_formatted['Category'].value_counts()

GDP (current US$)               266
GDP per capita (current US$)    266
Population, total               266
Name: Category, dtype: int64

In [124]:
# Paring down the countries contain to match the all_co_melted
pared_gdp_df = all_gdp_df_formatted[all_gdp_df_formatted['Country Code'].isin(all_co_melted['Country Code'])]

In [125]:
# Melt the data frame, id_vars is what columns to keep, value_vars is what columns to melt
gdp_melted = pd.melt(pared_gdp_df, id_vars=['Country', 'Country Code', 'Type', 'Category'],
                     var_name='Year', value_name='Value')

gdp_melted.head()

,Country,Country Code,Type,Category,Year,Value
0,Afghanistan,AFG,GDP,GDP (current US$),1960,537777811.111111
1,Afghanistan,AFG,GDP,GDP per capita (current US$),1960,62.3693745050559
2,Albania,ALB,GDP,GDP (current US$),1960,..
3,Albania,ALB,GDP,GDP per capita (current US$),1960,..
4,Algeria,DZA,GDP,GDP (current US$),1960,2723593384.78054


In [126]:
gdp_melted = gdp_melted[gdp_melted['Country'] != 'World']
gdp_melted = gdp_melted[gdp_melted['Country'] != 'International transport']

In [127]:
gdp_melted.isnull().sum()

Country         0
Country Code    0
Type            0
Category        0
Year            0
Value           0
dtype: int64

In [128]:
gdp_melted.dtypes

Country         object
Country Code    object
Type            object
Category        object
Year            object
Value           object
dtype: object

In [129]:
# Converting year to Int and GDP_Value to float, replace ".." with NaN
gdp_melted[['Year']] = gdp_melted[['Year']].astype(int).copy()
gdp_melted['Value'] = pd.to_numeric(gdp_melted['Value'], errors='coerce').copy()


In [130]:
# Shortening the time frame from 1999-2019
gdp_melted_short = gdp_melted[gdp_melted['Year'] >= 1999]

In [131]:
# reorder the columns
gdp_melted_ready = gdp_melted_short[['Country', 'Country Code', 'Year', 'Type', 'Category', 'Value']]

In [132]:
gdp_melted_ready.head()

,Country,Country Code,Year,Type,Category,Value
24102,Afghanistan,AFG,1999,GDP,GDP (current US$),NaN
24103,Afghanistan,AFG,1999,GDP,GDP per capita (current US$),NaN
24104,Albania,ALB,1999,GDP,GDP (current US$),3.212122e+09
24105,Albania,ALB,1999,GDP,GDP per capita (current US$),1.033243e+03
24106,Algeria,DZA,1999,GDP,GDP (current US$),4.864065e+10


### Make sure the CO2 df and GDP df have the same countries in them

In [133]:
# Get unique country codes for each df and compare
co_codes = all_co_ready['Country Code'].unique() 
gdp_codes = gdp_melted_ready['Country Code'].unique()

# Are there any countries in the CO2 df that are not in the per capita df?
co_not_in_gdp = []
for code in co_codes:
    if code not in gdp_codes:
        co_not_in_gdp.append(code)
print(f"There are {len(co_not_in_gdp)} countries in the CO2 df that are not in the GDP df.\
      The country not in the other data frame is: {co_not_in_gdp}")

# Show the countries that are not in the per capita df
co_not_in_gdp

# Are there any countries in the per capita df that are not in the CO2 df?
gdp_not_in_co = []   
for code in gdp_codes:
    if code not in co_codes:
        gdp_not_in_co.append(code)   
print(f"There are {len(gdp_not_in_co)} countries in the GDP df that are not in the CO2 df.\
      The country not in the other data frame is: {gdp_not_in_co}")

There are 20 countries in the CO2 df that are not in the GDP df.      The country not in the other data frame is: ['AIA', 'ATA', 'BES', 'CXR', 'COK', 'PCZ', 'GUF', 'GLP', 'KSV', 'MTQ', 'MYT', 'MSR', 'NIU', 'REU', 'SHN', 'SPM', 'TWN', 'WLF', 'XIT', 'WLD']
There are 0 countries in the GDP df that are not in the CO2 df.      The country not in the other data frame is: []


In [134]:
# Drop the countries in the CO2 df that are not in the GDP df
all_co_ready = all_co_ready[~all_co_ready['Country Code'].isin(co_not_in_gdp)]


### Merge the CO2 and GDP dataframes

In [135]:
all_co_ready.head()

,Country,Country Code,Year,Type,Category,Value
249,Afghanistan,AFG,1999,Total CO2,Total,1.091640
250,Afghanistan,AFG,2000,Total CO2,Total,1.047128
251,Afghanistan,AFG,2001,Total CO2,Total,1.069098
252,Afghanistan,AFG,2002,Total CO2,Total,1.340995
253,Afghanistan,AFG,2003,Total CO2,Total,1.559602


In [136]:
gdp_melted_ready.head()

,Country,Country Code,Year,Type,Category,Value
24102,Afghanistan,AFG,1999,GDP,GDP (current US$),NaN
24103,Afghanistan,AFG,1999,GDP,GDP per capita (current US$),NaN
24104,Albania,ALB,1999,GDP,GDP (current US$),3.212122e+09
24105,Albania,ALB,1999,GDP,GDP per capita (current US$),1.033243e+03
24106,Algeria,DZA,1999,GDP,GDP (current US$),4.864065e+10


In [137]:
# Merge the CO2 and GDP data frames
co_gdp_df = pd.merge(all_co_ready, gdp_melted_ready, how='outer', on=['Country', 'Country Code', 'Year', 'Type', 'Category', 'Value'])

In [138]:
co_gdp_df.head()

,Country,Country Code,Year,Type,Category,Value
0,Afghanistan,AFG,1999,Total CO2,Total,1.091640
1,Afghanistan,AFG,2000,Total CO2,Total,1.047128
2,Afghanistan,AFG,2001,Total CO2,Total,1.069098
3,Afghanistan,AFG,2002,Total CO2,Total,1.340995
4,Afghanistan,AFG,2003,Total CO2,Total,1.559602


In [139]:
# Test
test = co_gdp_df[co_gdp_df['Country Code'] == 'USA']

test['Category'].value_counts()

Other                           46
Oil                             46
Coal                            46
Cement                          46
Total                           46
Gas                             46
Flaring                         46
GDP (current US$)               21
GDP per capita (current US$)    21
Population, total               21
Name: Category, dtype: int64

#### Bringing in continent data onto the CO2/GDP/POP dataframe

In [140]:
cont_df = pd.read_csv('data/country-and-continent-codes-list-csv.csv')

cont_df.head()

,Continent_Name,Continent_Code,Country_Name,Two_Letter_Country_Code,Three_Letter_Country_Code,Country_Number
0,Asia,AS,"Afghanistan, Islamic Republic of",AF,AFG,4.0
1,Europe,EU,"Albania, Republic of",AL,ALB,8.0
2,Antarctica,AN,Antarctica (the territory South of 60 deg S),AQ,ATA,10.0
3,Africa,AF,"Algeria, People's Democratic Republic of",DZ,DZA,12.0
4,Oceania,OC,American Samoa,AS,ASM,16.0


In [141]:
print("Missing values in 'Two_Letter_Country_Code' column:", cont_df['Two_Letter_Country_Code'].isna().sum())


Missing values in 'Two_Letter_Country_Code' column: 1


In [142]:
# Drop the rows with missing values
cont_df = cont_df.dropna(subset=['Two_Letter_Country_Code'])

In [143]:
# Rename the columns to match the CO2 data
cont_df = cont_df.rename(columns = {'Three_Letter_Country_Code':'Country Code', 'Continent_Name':'Continent', 'Continent_Code':'Continent Code'})

In [144]:
# Check for duplicate country codes in the continent data frame
cont_df['Country Code'].value_counts()
# Drop the duplicate country codes
cont_df = cont_df.drop_duplicates(subset=['Country Code'])

In [145]:
cont_df.head()

,Continent,Continent Code,Country_Name,Two_Letter_Country_Code,Country Code,Country_Number
0,Asia,AS,"Afghanistan, Islamic Republic of",AF,AFG,4.0
1,Europe,EU,"Albania, Republic of",AL,ALB,8.0
2,Antarctica,AN,Antarctica (the territory South of 60 deg S),AQ,ATA,10.0
3,Africa,AF,"Algeria, People's Democratic Republic of",DZ,DZA,12.0
4,Oceania,OC,American Samoa,AS,ASM,16.0


In [146]:
print("Columns in co_gdp_df:", co_gdp_df.columns)
print("Columns in cont_df:", cont_df.columns)

Columns in co_gdp_df: Index(['Country', 'Country Code', 'Year', 'Type', 'Category', 'Value'], dtype='object')
Columns in cont_df: Index(['Continent', 'Continent Code', 'Country_Name',
       'Two_Letter_Country_Code', 'Country Code', 'Country_Number'],
      dtype='object')


In [147]:
# Bringing in continent data on to the co2/gdp/pop data frame, only bringing the Continent_Name column and the continent_Code column 
co_gdp_cont_df = pd.merge(co_gdp_df, cont_df[['Continent', 'Continent Code', 'Country Code']], how='left', left_on='Country Code', right_on='Country Code')


In [148]:
co_gdp_cont_df.head()

,Country,Country Code,Year,Type,Category,Value,Continent,Continent Code
0,Afghanistan,AFG,1999,Total CO2,Total,1.091640,Asia,AS
1,Afghanistan,AFG,2000,Total CO2,Total,1.047128,Asia,AS
2,Afghanistan,AFG,2001,Total CO2,Total,1.069098,Asia,AS
3,Afghanistan,AFG,2002,Total CO2,Total,1.340995,Asia,AS
4,Afghanistan,AFG,2003,Total CO2,Total,1.559602,Asia,AS


In [149]:
# Find where continet code is null
missing_conts = co_gdp_cont_df[co_gdp_cont_df['Continent Code'].isnull()]
missing_conts['Continent'].value_counts()


North America    12257
Name: Continent, dtype: int64

In [150]:
# Fill the Continent Code column for all North America countries with NA
co_gdp_cont_df['Continent Code'] = co_gdp_cont_df['Continent Code'].fillna('NA')

### Import to MongoDB

In [151]:

# These are the two data frames that are ready to be loaded into MongoDB one with all the countries, their continent info, CO2, GDP, and population data, and the 
# other with all the countries, continents, and geojson data.

# co_gdp_cont_df - all co2, gdp, pop, and continent data


In [152]:
# Convert the dataframes to a list of dictionaries
co_gdp_cont_df = co_gdp_cont_df.to_dict('records')


In [153]:
# connect to the MongoDB client and create the database and collection
client = MongoClient('mongodb://localhost:27017/')
db = client['global_emissions_db']
collection = db['CO2_gdp_population']

# Insert the data into the collection
collection.insert_many(co_gdp_cont_df)


### Test the Database   

In [154]:
# Create a decorator to manage the connection to the MongoDB server
def with_mongo_client(func):
    def wrapper(*args, **kwargs):
        client = MongoClient("mongodb://localhost:27017/")
        try:
            return func(client, *args, **kwargs)
        finally:
            client.close()

    return wrapper

# Test fuctions for the decorator and databse 

@with_mongo_client
def list_all_collections(client):
    db = client['global_emissions_db']
    return db.list_collection_names()


@with_mongo_client
def find_five_countries_documents(client):
    db = client['global_emissions_db']
    collection = db['CO2_gdp_population']
    return list(collection.find().limit(5))

# list all collections
@with_mongo_client
def list_all_collections(client):
    db = client['global_emissions_db']
    return db.list_collection_names()

In [155]:
find_five_countries_documents()

[{'_id': ObjectId('6437477f3aadaf090b0352e5'),
  'Country': 'Afghanistan',
  'Country Code': 'AFG',
  'Year': 1999,
  'Type': 'Total CO2',
  'Category': 'Total',
  'Value': 1.09164,
  'Continent': 'Asia',
  'Continent Code': 'AS'},
 {'_id': ObjectId('6437477f3aadaf090b0352e6'),
  'Country': 'Afghanistan',
  'Country Code': 'AFG',
  'Year': 2000,
  'Type': 'Total CO2',
  'Category': 'Total',
  'Value': 1.047128,
  'Continent': 'Asia',
  'Continent Code': 'AS'},
 {'_id': ObjectId('6437477f3aadaf090b0352e7'),
  'Country': 'Afghanistan',
  'Country Code': 'AFG',
  'Year': 2001,
  'Type': 'Total CO2',
  'Category': 'Total',
  'Value': 1.0690979999999999,
  'Continent': 'Asia',
  'Continent Code': 'AS'},
 {'_id': ObjectId('6437477f3aadaf090b0352e8'),
  'Country': 'Afghanistan',
  'Country Code': 'AFG',
  'Year': 2002,
  'Type': 'Total CO2',
  'Category': 'Total',
  'Value': 1.3409950000000002,
  'Continent': 'Asia',
  'Continent Code': 'AS'},
 {'_id': ObjectId('6437477f3aadaf090b0352e9'),
  

In [156]:
list_all_collections()

['CO2_gdp_population']

In [160]:
client = MongoClient('mongodb://localhost:27017/')
db = client['global_emissions_db']

# How many countries are in the country_geojson_data collection?
db.CO2_gdp_population.count_documents({})


79247

In [158]:
# client = MongoClient('mongodb://localhost:27017/')
# db = client['global_emissions_db']

# # Delete all the collections
# for collection_name in db.list_collection_names():
#     db[collection_name].drop() 